In [2]:
import pandas as pd
import numpy as np
import os

# ==========================================
# PART 1: CONFIGURATION
# ==========================================
THRESHOLDS = {
    'ZERO_CONV_THRESHOLD': 50.0,   # Max spend allowed with 0 conversions before flagging
    'ZOMBIE_ROAS': 1.0,            # Minimum acceptable ROAS (Revenue / Spend)
    'ZOMBIE_SPEND_MIN': 100.0      # Minimum spend to qualify as a "Zombie"
}

# Map your CSV headers to these standard names
COLUMN_MAPPING = {
    'Day': 'date',
    'Campaign': 'campaign_name',
    'Cost': 'spend',
    'Impressions': 'impressions',
    'Clicks': 'clicks',
    'Conversions': 'conversions',
    'Total conv. value': 'revenue'
}

# ==========================================
# PART 2: INGESTION
# ==========================================
def load_data(filename):
    print(f"Loading data from: {filename}...")
    if not os.path.exists(filename):
        print(f"ERROR: Could not find {filename}. Make sure it is in this folder.")
        return None
        
    try:
        df = pd.read_csv(filename)
    except Exception as e:
        print(f"ERROR reading CSV: {e}")
        return None
        
    # Rename columns based on mapping
    df = df.rename(columns=COLUMN_MAPPING)
    
    # Check for required columns
    required_cols = ['campaign_name', 'spend', 'conversions', 'revenue']
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        print(f"ERROR: Missing required columns: {missing}")
        print(f"Current columns found: {list(df.columns)}")
        return None

    if 'date' in df.columns:
        # 'mixed' allows handling 17/12/24 OR 2024-12-17 automatically
        df['date'] = pd.to_datetime(df['date'], format='mixed', dayfirst=True)
        
    return df

# ==========================================
# PART 3: ANALYSIS
# ==========================================
def analyze_campaigns(df):
    print("Aggregating data and running audit logic...")
    
    # 1. Aggregate Daily Data into Campaign Totals
    # We use 'sum' for metrics to get lifetime stats from the daily rows
    camp_stats = df.groupby('campaign_name').agg({
        'spend': 'sum',
        'conversions': 'sum',
        'revenue': 'sum',
        'clicks': 'sum'
    }).reset_index()

    # 2. Calculate KPIs (Math)
    # np.where is used to prevent "Divide by Zero" errors
    camp_stats['roas'] = np.where(camp_stats['spend'] > 0, camp_stats['revenue'] / camp_stats['spend'], 0)
    camp_stats['cpa'] = np.where(camp_stats['conversions'] > 0, camp_stats['spend'] / camp_stats['conversions'], 0)

    # 3. Apply Logic Flags (The "Audit")
    
    # Flag 1: ZERO HOPE
    # Campaigns that have spent significant money but produced 0 conversions.
    camp_stats['is_zero_hope'] = (camp_stats['spend'] > THRESHOLDS['ZERO_CONV_THRESHOLD']) & (camp_stats['conversions'] == 0)
    
    # Flag 2: ZOMBIES
    # Campaigns that are getting conversions, but doing so unprofitably (ROAS < 1.0) 
    # and have spent enough to be statistically significant.
    camp_stats['is_zombie'] = (
        (camp_stats['spend'] > THRESHOLDS['ZOMBIE_SPEND_MIN']) & 
        (camp_stats['roas'] < THRESHOLDS['ZOMBIE_ROAS']) & 
        (camp_stats['conversions'] > 0)
    )
    
    return camp_stats

# ==========================================
# PART 4: INSIGHT TRANSLATION
# ==========================================
def generate_insights(df):
    insights = []
    
    zero_hope = df[df['is_zero_hope'] == True]
    zombies = df[df['is_zombie'] == True]
    
    wasted_spend = zero_hope['spend'].sum()
    inefficient_spend = zombies['spend'].sum()
    total_bleed = wasted_spend + inefficient_spend
    
    insights.append(f"Total Opportunity Identified: ${total_bleed:,.2f}")
    
    if wasted_spend > 0:
        insights.append(f"• CRITICAL: ${wasted_spend:,.2f} wasted on {len(zero_hope)} campaigns with ZERO conversions.")
        
    if inefficient_spend > 0:
        insights.append(f"• WARNING: ${inefficient_spend:,.2f} detected in 'Zombie' campaigns (ROAS < 1.0).")

    # Find worst offender to highlight
    bad_campaigns = pd.concat([zero_hope, zombies])
    if not bad_campaigns.empty:
        # Sort by highest spend to find the biggest problem
        worst_offender = bad_campaigns.sort_values(by='spend', ascending=False).iloc[0]
        
        fix_msg = (
            f"\n[FIX THIS FIRST]\n"
            f"Campaign: '{worst_offender['campaign_name']}'\n"
            f"Problem: Spent ${worst_offender['spend']:,.2f} with ROAS of {worst_offender['roas']:.2f}.\n"
            f"Action: Pause immediately."
        )
        insights.append(fix_msg)
        
    return insights

# ==========================================
# PART 5: EXECUTION (The "Main" Loop)
# ==========================================
if __name__ == "__main__":
    # 1. Define your filename here
    FILENAME = 'big_google_ads_data.csv' 
    
    # 2. Run the pipeline
    df_raw = load_data(FILENAME)
    
    if df_raw is not None:
        # Run the audit
        audit_results = analyze_campaigns(df_raw)
        
        # Generate and print text insights
        print("\n" + "="*30)
        print(" AUDIT REPORT ")
        print("="*30)
        
        text_report = generate_insights(audit_results)
        for line in text_report:
            print(line)
            
        # 3. EXPORT: Save the bad campaigns to a file for review
        # Filter for only the bad ones
        bad_campaigns = audit_results[
            (audit_results['is_zero_hope'] == True) | 
            (audit_results['is_zombie'] == True)
        ].copy()
        
        if not bad_campaigns.empty:
            output_file = 'audit_results_BAD_CAMPAIGNS.xlsx'
            try:
                bad_campaigns.to_excel(output_file, index=False)
                print(f"\n[ACTION] Detailed list of {len(bad_campaigns)} bad campaigns saved to: {output_file}")
            except Exception as e:
                print(f"\n[ERROR] Could not save Excel file. Is it open? Error: {e}")
        else:
            print("\n[GOOD NEWS] No bad campaigns found!")

Loading data from: big_google_ads_data.csv...
Aggregating data and running audit logic...

 AUDIT REPORT 
Total Opportunity Identified: $1,452,814.27
• CRITICAL: $410,166.44 wasted on 14 campaigns with ZERO conversions.
• WARNING: $1,042,647.83 detected in 'Zombie' campaigns (ROAS < 1.0).

[FIX THIS FIRST]
Campaign: 'Campaign_042_ZOMBIE'
Problem: Spent $55,630.68 with ROAS of 0.50.
Action: Pause immediately.

[ACTION] Detailed list of 35 bad campaigns saved to: audit_results_BAD_CAMPAIGNS.xlsx


In [4]:
pip install streamlit plotly pandas

Note: you may need to restart the kernel to use updated packages.
